In [0]:
from google.colab import drive
from zipfile import ZipFile
import torch
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# read config
config
with open("../config.json") as f:
  config = json.load(f)["locations"]

In [0]:
# Unzips the patents zip

zippedFile = config["zipped_patents"]
with ZipFile(zippedFile, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [0]:
all_text = []
folder = "patent_text/"
files = [f"{x}" for x in os.listdir(folder)]
for i, file in enumerate(files):
  if i % 5000 == 0 and i > 1:
    print(i)
  with open("patent_text/"+file) as f:
    lines = [line.strip("\n ") for line in f.readlines()]
    all_text.append(lines)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000


In [0]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [0]:
  def get_embeddings_from_file(file_name):
    with open(f"patent_text/{file_name}", "r") as f:
      lines = [line.strip("\n ") for line in f.readlines()]
      try:
        sentence_embeddings = (
            embed.predict(tf.constant(lines))["outputs"]
              .numpy()
              .mean(axis=0)
              .reshape((512, 1))
        )
      except:
        sentence_embeddings = np.zeros((500,1))
      return sentence_embeddings

In [0]:
folder = "patent_text/"
files = {idx: f"{file_name}" for idx, file_name in enumerate(os.listdir(folder))}
start_idx, end_idx = 0, 30000
embeddings = np.empty(end_idx-start_idx, dtype=object)
# embeddings = np.array([])
print(embeddings.shape)
i = 0
for idx in tqdm(range(start_idx, end_idx)):
  file = files[idx]
  if idx % 5000 == 0 and idx > 0:
    print(idx)
    # print("result: ", result)
    # print("embeddings length", len(embeddings))
    df = pd.DataFrame({
          "patents": [files[x] for x in range(start_idx, idx+1)], 
          "embedding": embeddings[:i+1]
        })
    df.to_json(config["universal_embeddings"])
  result = get_embeddings_from_file(file) 
  embeddings[i] = result
  i += 1

df = pd.DataFrame({
      "patents": [files[x] for x in range(start_idx, end_idx)], 
      "embedding": embeddings
    })

  0%|          | 0/30000 [00:00<?, ?it/s]

(30000,)


 17%|█▋        | 4999/30000 [06:46<22:43, 18.34it/s]

5000


 33%|███▎      | 9999/30000 [14:50<34:49,  9.57it/s]

...

 67%|██████▋   | 20000/30000 [34:49<26:53,  6.20it/s]

20000


 83%|████████▎ | 24999/30000 [46:07<05:58, 13.96it/s]

25000


100%|██████████| 30000/30000 [57:38<00:00,  8.68it/s]


In [0]:
df.to_json(config["universal_embeddings"])